Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Período a consultar

In [3]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 1).date()
data_fim = datetime(2023, 12, 8).date()
# data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
# data_fim = data_fim - timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print("data_inicio: ", data_inicio)
print("data_fim: ", data_fim)

data_inicio:  2023-11-01
data_fim:  2023-12-08


### Historico de estoque fulfillment

In [4]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-11-01' AND '2023-12-08;'


C:\Users\couti\AppData\Local\Temp\ipykernel_6272\2016158527.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [5]:
# df_stock

In [6]:
# print(df_stock.shape)
# duplicates = df_stock[df_stock.duplicated(keep=False)]
# duplicates

In [7]:
# # datas consultadas, dias em que um produto pode ou não estar disponível
# # df_stock["created_at"].value_counts().index.to_list()
# df_stock["created_at"].value_counts().sort_index()
# df_stock["created_at"].dt.date.value_counts().sort_index()
# # df_stock["created_at"].dt.date.value_counts()

# # len(df_stock["created_at"].dt.date.value_counts().sort_index())

In [8]:
# df_stock

In [9]:
# # buscando anuncio e verificando dias consultados
# result = df_stock[df_stock['ml_inventory_id'] == 'FBAY80623']
# result = result.sort_values(by='created_at')
# result

In [10]:
# # encontrando anuncios pela data
# df_stock["created_at"] = pd.to_datetime(df_stock["created_at"])
# result = df_stock.loc[df_stock['created_at'].dt.date == pd.to_datetime('2023-12-7').date()]
# result

In [11]:
# df_stock[df_stock['created_at']=='2023-12-07 03:01:03.036386']

In [12]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

In [13]:
# df_stock

In [14]:
# df_stock.dtypes

In [15]:
# df_stock['data'] = pd.to_datetime(df_stock['data'])

# dfx = condf(df_stock, 'data', '2023-11-1')
# dfx

In [16]:
# # A linha abaixo conta as ocorrências de cada valor em 'ml_inventory_id'
# # value_counts = result['ml_inventory_id'].value_counts()
# value_counts = df_stock['ml_inventory_id'].value_counts()

# # Filtra para incluir apenas aqueles que ocorrem mais de 2 vezes
# filtered_counts = value_counts[value_counts > 1]

# filtered_counts.shape
# # filtered_counts
# value_counts

In [17]:
# df_stock[df_stock["detail_status"] == "transfer"]

In [18]:
# condf(df_stock,'ml_inventory_id','CRMK90073')

In [19]:
# condf(df_stock, "detail_status", "transfer")

In [20]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)

In [21]:
# df_stock

In [22]:
# condf(df_stock, "ml_inventory_id", "DSGP06967")

In [23]:
# print(df_stock.shape)
# df_stock = df_stock.drop_duplicates()
# print(df_stock.shape)

#### Dias em que produto esteve disponível

In [24]:
# condf(df_stock,'ml_inventory_id','ABCB20467')

In [25]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})

In [26]:
# days_available

In [27]:
# days_available.sort_values(by='days_available',ascending=False)
# days_available['days_available'].value_counts()

# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [28]:
# condf(df_stock, "ml_inventory_id", "MENA66694")

In [29]:
# ## Contando dias em que produto esteve disponível
# days_available_ = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available_ = days_available_.rename(columns={"has_stock": "days_available"})
# days_available_

In [30]:
# condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(days_available, "ml_inventory_id", "DSGP06967")

In [34]:
# Unindo DFs
df_stock_days = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock_days_left = df_stock.merge(days_available, on=["ml_inventory_id"], how="left")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
# print(df_stock.shape)
#

In [35]:
# print(df_stock.shape)
# print(df_stock_days.shape)
# print(df_stock_days_left.shape)
# df_stock_days.sample(3)

(13104, 8)
(13104, 9)
(13104, 9)


,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
3026,DHEV27121,2,None,NaN,MLB2003287174,0,2023-12-05,True,36
1484,BOWK48319,7,None,NaN,MLB1461189878,51907695787,2023-11-29,True,36
1970,SYZS70682,0,None,NaN,MLB1729421535,69642779895,2023-11-11,False,0


In [ ]:
# # Unindo DFs
# df_stock_ = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# # df_stock_.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# # df_stock_ = df_stock_.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# # Verificando o formato após a correção
# print(df_stock_.shape)
# df_stock_

In [39]:
# data de hoje
# data_de_hoje = datetime.now().date() - timedelta(days=1)
# print(data_de_hoje)
data_de_hoje = datetime.now().date()
data_de_hoje = data_de_hoje - timedelta(days=1)

print(data_de_hoje)
df_stock_days["data"] = pd.to_datetime(df_stock_days["data"])
# data_de_hoje = data_de_hoje + timedelta(days=1)  # + 1 dia para pegar a data atual no DB

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock_days[df_stock_days["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)
# df_stock_today = df_stock.drop(['has_stock'], axis=1)

2023-12-07


In [40]:
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
0,HXJD21466,0,None,NaN,MLB1401964928,0,2023-12-07,False,0
36,QCVC39144,0,None,NaN,MLB2003129630,0,2023-12-07,False,0
72,SPKV63227,0,None,NaN,MLB1490210934,0,2023-12-07,False,0
108,PQCQ36986,13,None,NaN,MLB2075516288,173887190665,2023-12-07,True,36
144,UITT86657,0,None,NaN,MLB1734684354,69991314080,2023-12-07,False,0
...,...,...,...,...,...,...,...,...,...
12924,FSNB97805,6,transfer,30.0,MLB1992889189,173551307782,2023-12-07,True,36
12960,FZQP10396,4,None,NaN,MLB2006333696,0,2023-12-07,True,36
12996,HNWI37644,5,None,NaN,MLB2003228786,0,2023-12-07,True,36
13032,AVOX05934,7,None,NaN,MLB2003190501,0,2023-12-07,True,36


In [54]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [55]:
# df_stock_today.sample(5)

In [56]:
# condf(df_stock,'ml_inventory_id','FSNB97805')
# condf(df_stock_today,'ml_inventory_id','FSNB97805')

### Buscando hitorico de orders no BD

In [57]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM ml_orders WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# # Ordenando orders por data
# df_orders = df_orders.sort_values(by="date_approved", ascending=False)
# df_orders["data"] = df_orders["date_approved"].dt.date
# df_orders = df_orders.drop(["date_closed","date_approved"], axis=1)

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

# df_orders = df_orders[df_orders["order_status"] == "paid"]
# df_orders = df_orders[df_orders["payment_status"] == "approved"]


# df_orders = df_orders.drop(
#     ["pack_id", "date_approved", "order_status", "payment_status"], axis=1
# )
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

SELECT * FROM ml_orders WHERE date_closed BETWEEN '2023-11-01' AND '2023-12-08'


C:\Users\couti\AppData\Local\Temp\ipykernel_6272\1893445520.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [58]:
print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
print(df_orders.shape)

(5352, 13)
(4612, 13)


In [81]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

In [83]:
df_orders.sample(3)

,ml_code,variation_id,seller_sku,order_id,sold_quantity,title,order_status,payment_status,shipping_id,variation_value_name,logistic_type,data
947,MLB1992541482,173552533264,FULLSG6684,2000006902197856,2,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,paid,approved,42802756417,0.09,fulfillment,2023-11-15
1275,MLB4200090836,0,FULLBASSOCLA13,2000006919368234,1,Alça Correia Basso Couro Preto Cla13 Guitarra ...,paid,approved,42810295232,None,fulfillment,2023-11-17
5295,MLB1435852259,0,FULLELIXIR3216,2000007019090540,1,Encordoamento Elixir .010 Light Nanoweb Para G...,paid,approved,42852922432,None,fulfillment,2023-11-29


In [59]:
# df_orders.head()

In [60]:
# df_orders
# df_orders['pack_id'].value_counts()

# # condf(df_orders,'pack_id','2000005059931419')

In [61]:
# print(df_orders.shape)
# df_orders.head(3)

In [62]:
# condf(df_orders, "ml_code", "MLB1992541482")

#### Total de vendas por ml_code e variation_id

In [63]:
# df_orders.shape

In [64]:
# df_orders

In [65]:
# condf(df_orders,'ml_code','MLB3516894673')

In [84]:
# Total de vendas por ml_code e id de variação
df_orders_quantity = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

In [85]:
df_orders_quantity

,ml_code,variation_id,sold_quantity
0,MLB1363602423,0,24
1,MLB1398114866,174301393967,1
2,MLB1398114866,174302125663,2
3,MLB1399101853,0,1
4,MLB1400355425,0,20
...,...,...,...
322,MLB4202595608,0,20
323,MLB4217370456,0,1
324,MLB4237823132,0,6
325,MLB924922735,0,14


In [86]:
print(f"Número de ml_code únicos: {len(df_orders_quantity['ml_code'].unique())}")
print(
    f"Número de variation_id únicos: {len(df_orders_quantity['variation_id'].unique())}"
)

Número de ml_code únicos: 256
Número de variation_id únicos: 128


In [87]:
# condf(df_orders_quantity,'ml_code','MLB2075516288')
# df_orders_quantity['ml_code'].value_counts()

,ml_code,variation_id,sold_quantity
132,MLB2075516288,173887022577,58
133,MLB2075516288,173887022578,23
134,MLB2075516288,173887022579,23
135,MLB2075516288,173887190664,8
136,MLB2075516288,173887190665,13
137,MLB2075516288,173887190666,1
138,MLB2075516288,173887190667,11


In [89]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    df_orders_quantity,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)

In [156]:
print(f"Total de vendas = {df_total_sales.shape[0]}")

Total de vendas = 4612


In [ ]:
# df_orders.shape

In [ ]:
# df_total_sales.columns

In [ ]:
# df_total_sales.sample()

In [ ]:
# # # Exibir todas as linhas e colunas
# # pd.set_option('display.max_rows', None)
# # pd.set_option('display.max_columns', None)

# # Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')


# # condf(df_total_sales,'ml_code','MLB3427769549')
# # # x = condf(df_orders,'ml_code','MLB3427769549')

# # # x = x.sort_values(by='data')
# # # x.head(15)
# # x = df_orders[df_orders['ml_code']=='MLB3427769549']
# # x['data'] = pd.to_datetime(x['data'])
# # x[x['data']=='2023-11-22']
# # # x['sold_quantity'].value_counts()

# # # counts = x[x['sold_quantity'] > 1]['sold_quantity'].value_counts()
# # # counts

# # y = x.groupby('data')['sold_quantity'].sum().reset_index()
# # y

# # # # x

In [ ]:
# x.dtypes

In [ ]:
# df_total_sales['data'].value_counts()

In [ ]:
# condf(df_total_sales, "ml_code", "MLB4127982748")
# condf(df_total_sales, "ml_code", "MLB4127982748")

In [ ]:
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
# df_total_sales = df_total_sales.drop_duplicates()

In [ ]:
# condf(df_total_sales, "ml_code", "MLB1992541482")

Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [ ]:
# print(df_total_sales.shape)
# df_total_sales.sample()

In [ ]:
# condf(df_total_sales, 'ml_code', 'MLB3461718736')
# condf(df_orders, 'ml_code', 'MLB3461718736')
# # condf(resultado, 'ml_code', 'MLB3427769549')
# # condf(df_stock, 'ml_code', 'MLB3427769549')

In [ ]:
# print(df_stock_today.shape)
# df_stock_today.sample()

#### Buscando Produtos

In [157]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)
    sql_query = "SELECT * FROM items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

C:\Users\couti\AppData\Local\Temp\ipykernel_6272\3517383442.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


In [158]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

In [159]:
df_not_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
5,MLB949776093,QYGO52123,NaN,NaN,paused,False


In [160]:
df_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
399,MLB3847886806,AJZG23550,NaN,NaN,active,True


In [161]:
df_not_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
3,MLB924922735,JFGN34621,NaN,NaN,paused,False
4,MLB949771924,TDFV00990,NaN,NaN,paused,False
5,MLB949776093,QYGO52123,NaN,NaN,paused,False
6,MLB949788598,ERIM51807,NaN,NaN,paused,False
7,MLB949790387,DGBX50777,NaN,NaN,paused,False
...,...,...,...,...,...,...
512,MLB1662862664,CRMK90073,Azul,64489869774,active,False
513,MLB1942106803,WQLG77218,nan,NaN,active,False
514,MLB1965265311,IPQB89761,nan,NaN,active,False
515,MLB1976555976,ZEWV91930,nan,NaN,active,False


In [162]:
print(df_not_catalogo.shape)
print(df_catalogo.shape)

(361, 6)
(167, 6)


In [119]:
df_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
0,MLB3778561802,AVOX05934,NaN,NaN,closed,True
1,MLB3436515963,WTHI04163,NaN,NaN,closed,True
2,MLB3778538730,JLAX10536,NaN,NaN,closed,True
47,MLB1459799426,SKXW47953,NaN,NaN,paused,True
136,MLB1848507503,IKDC16959,NaN,NaN,paused,True
...,...,...,...,...,...,...
523,MLB3497394073,XGSW66673,nan,NaN,active,True
524,MLB3516894673,PPWL59058,nan,NaN,active,True
525,MLB3513621863,ABCB20467,nan,None,active,True
526,MLB4272734766,ENML56386,nan,None,active,True


In [100]:
# df_orders.sample()

In [172]:
df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="left",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="left",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

In [173]:
df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].fillna(0)
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].fillna(0)

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].astype("int64")
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].astype("int64")

In [174]:
# df_total_sales_cat.sample(5)
# df_total_sales_not_cat['total_sold_quantity'].value_counts()
# df_total_sales_cat['total_sold_quantity'].value_counts()

In [175]:
print(df_total_sales_cat.shape)
print(df_total_sales_not_cat.shape)

(2730, 15)
(1631, 14)


In [176]:
# df_total_sales_cat

In [180]:
# df_total_sales_cat = df_total_sales_cat.drop(
#     columns=["data", "shipping_id", "variation_id_x", "order_id"]
# )
df_total_sales_cat_x = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat_x = df_total_sales_cat_x.drop_duplicates()
df_total_sales_cat_x.shape

(168, 11)

In [170]:
# df_total_sales_cat_x

In [145]:
# df_total_sales_cat['value_name'].value_counts()

In [146]:
# condf(df_total_sales_cat, 'ml_code','MLB3427769549')

In [147]:
# df_vendas_total_sales_not_cat

In [148]:
print(df_total_sales_not_cat.shape)

(1631, 14)


In [181]:
print(df_total_sales_not_cat.shape)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat_x.drop_duplicates()
print(df_total_sales_not_cat_x.shape)

(1631, 14)
(361, 11)


In [183]:
# df_total_sales_cat_x

In [184]:
print(df_total_sales_cat_x.shape)
print(df_total_sales_not_cat_x.shape)

(168, 11)
(361, 11)


In [153]:
# df_total_sales_cat['ml_inventory_id'].value_counts()

In [186]:
# Antes de desabilitar
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Seu código para exibir o DataFrame

# # Depois de desabilitar
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')


# df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_cat_x = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

In [188]:
# df_orders.shape

In [189]:
# condf(df_total_sales_cat,'ml_inventory_id','WTHI04163')
# condf(y,'ml_inventory_id','WTHI04163')

In [190]:
# condf(df_total_sales_cat,'ml_inventory_id','AWAZ99732')

In [210]:
df_total_cat = df_total_sales_cat_x.copy()
df_total_not_cat = df_total_sales_not_cat_x.copy()

In [211]:
df_total_cat.columns

Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id_x', 'status',
       'catalog_listing', 'variation_id_y', 'seller_sku', 'order_id', 'title',
       'shipping_id', 'variation_value_name', 'logistic_type', 'data',
       'total_sold_quantity'],
      dtype='object')

In [212]:
# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat,
    df_sum_qt_sold_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})


df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat,
    df_sum_qt_sold_not_cat[["ml_inventory_id", "total_sold_not_catalog"]],
    on="ml_inventory_id",
    how="left",
)

In [215]:
print(df_total_cat.shape)
print(df_total_not_cat.shape)

(361, 15)

In [219]:
df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)
df_total_cat.shape

(166, 16)

In [217]:
# condf(df_total_cat,'ml_inventory_id','WTHI04163')

In [220]:
df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

# df_total_cat[df_total_cat['ml_inventory_id']=='WTHI04163']

pergar ml_inventory_id e sum_total e acrescesntar essas informações no df de não catalogo pelo ml_inventory_id
ao fim deixar somatório do catalogo e não catalogo


In [197]:
# cols=['ml_inventory_id','total_sold_catalog']
# df_total_cat = df_total_cat[cols]
# df_total_cat.sample()

In [221]:
df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)
# df_combined.sample()

In [ ]:
# df_total_not_cat.shape

In [ ]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [ ]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [ ]:
# df_combined['ml_inventory_id'].value_counts()

In [223]:
print(df_combined.shape)

(361, 16)


In [229]:
df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="left")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

In [230]:
df.shape

(361, 25)

In [237]:
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]
# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["total_available_quantity"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)

In [236]:
df.columns

Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id', 'status',
       'catalog_listing', 'seller_sku', 'order_id', 'title', 'shipping_id',
       'variation_value_name', 'logistic_type', 'data_x',
       'total_sold_quantity', 'total_sold_not_catalog', 'total_sold_catalog',
       'available_quantity_today', 'detail_status', 'detail_quantity',
       'references_id', 'references_variation_id', 'data_y', 'has_stock',
       'days_available', 'total_available_quantity', 'total_sold',
       'period_send_fulfillment'],
      dtype='object')

In [232]:
df.shape

(361, 27)

In [255]:
df_have_itens = df[df["days_available"] > 0]
df_have_itens.shape

(252, 27)

In [235]:
# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])
df_no_itens.shape

(109, 26)

In [256]:
df_sold_zero = df_have_itens[df_have_itens["total_sold"] == 0]
df_sold = df_have_itens[df_have_itens["total_sold"] > 0]

In [257]:
df_sold_zero.sample(3)

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,seller_sku,order_id,title,shipping_id,...,detail_status,detail_quantity,references_id,references_variation_id,data_y,has_stock,days_available,total_available_quantity,total_sold,period_send_fulfillment
28,MLB1417658704,ZZRV30730,NaN,NaN,active,False,NaN,NaN,NaN,NaN,...,None,NaN,MLB1417658704,0,2023-12-07,True,36,5,0,-5.0
119,MLB1712597299,JOGC89676,NaN,NaN,active,False,NaN,NaN,NaN,NaN,...,None,NaN,MLB1712597299,0,2023-12-07,True,36,2,0,-2.0
255,MLB1440243016,YAZB38863,NaN,NaN,active,False,NaN,NaN,NaN,NaN,...,None,NaN,MLB1440243016,0,2023-12-07,True,36,1,0,-1.0


In [258]:
df_sold.sample(3)

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,seller_sku,order_id,title,shipping_id,...,detail_status,detail_quantity,references_id,references_variation_id,data_y,has_stock,days_available,total_available_quantity,total_sold,period_send_fulfillment
335,MLB3696018282,CTZK75126,1,178531159091,active,False,FULLVANDORENSR261,2000006952695020,"2 Palheta Sax Alto Vandoren Java Origina 1 1,5...",42824045873,...,None,NaN,MLB3696018282,178531159091,2023-12-07,True,36,1,1,-0.0
263,MLB2216644379,LUHT78439,"1,5",17435050308,active,False,FULLVANDORENSR2615,2000006920197298,"2 Palheta Sax Alto Vandoren Java Origina 1 1,5...",42810649590,...,transfer,2.0,MLB2216644379,174350503080,2023-12-07,True,36,19,2,-17.0
265,MLB2216644379,FRBW82060,"2,5",174350503082,active,False,FULLVANDORENSR2625,2000007025520574,"2 Palheta Sax Alto Vandoren Java Origina 1 1,5...",42855620339,...,None,NaN,MLB2216644379,174350503082,2023-12-07,True,36,20,4,-16.0


In [239]:
dfx = df_have_itens.copy()

In [240]:
# dfx.columns

In [246]:
cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "total_available_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
]

dfx = dfx[cols]

In [249]:
dfx = dfx.rename(
    columns={"detail_status": "transfer_status", "detail_quantity": "transfer_quantity"}
)

In [252]:
dfx.sample()
dfx.shape

(252, 15)

## Acabou!